In [103]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import tensorflow as tf
from tensorflow import keras
from keras import initializers
from keras.layers import Input
from keras.models import Model, Sequential
from keras.layers.core import Dense, Dropout
from keras.layers import LeakyReLU
from keras.datasets import mnist
from keras.optimizers import Adam

In [104]:
def load_dataset():
  (x_train,y_train),(x_test,y_test)=mnist.load_data()
  x_train=(x_train.astype(np.float32)-127.5)/127.5 #normalize input to be [-1,1]
  x_train=x_train.reshape(60000,784) #reshape the data 
  return (x_train,y_train,x_test,y_test)

In [105]:
random_dim = 100

In [106]:
optimizer=keras.optimizers.Adam(lr=0.0002, beta_1=0.5)

In [107]:
loss=tf.keras.losses.BinaryCrossentropy()

In [108]:
def Generator(optimizer):
  generator=Sequential()
  generator.add(Dense(256,input_dim=random_dim, kernel_initializer=keras.initializers.RandomNormal(stddev=0.02)))
  generator.add(LeakyReLU(0.2))

  generator.add(Dense(512))
  generator.add(LeakyReLU(0.2))

  generator.add(Dense(1024))
  generator.add(LeakyReLU(0.2))

  generator.add(Dense(784, activation='tanh'))
  generator.compile(loss=loss, optimizer=optimizer)
  return generator

In [109]:
def Discriminator(optimizer):
    discriminator = Sequential()
    discriminator.add(Dense(1024, input_dim=784, kernel_initializer=keras.initializers.RandomNormal(stddev=0.02)))
    discriminator.add(LeakyReLU(0.2))
    discriminator.add(Dropout(0.3))

    discriminator.add(Dense(512))
    discriminator.add(LeakyReLU(0.2))
    discriminator.add(Dropout(0.3))

    discriminator.add(Dense(256))
    discriminator.add(LeakyReLU(0.2))
    discriminator.add(Dropout(0.3))

    discriminator.add(Dense(1, activation='sigmoid'))
    discriminator.compile(loss=loss, optimizer=optimizer)
    return discriminator

In [111]:
def model(generator, discriminator, optimizer, random_dim):   #put the model together to get gan
    gan_input=Input(shape=(random_dim,)) #noise input as 100-dimensional vectors
    x=generator(gan_input) #get the output set of the generator
    gan_output=discriminator(x) #get the whole output of the D
    GAN=Model(inputs=gan_input, outputs=gan_output)
    GAN.compile(loss=loss, optimizer=optimizer)
    return GAN

In [112]:
def plot_generated_images(epoch, generator, examples=100, dim=(10, 10), figsize=(10, 10)):
    noise = np.random.normal(0, 1, size=[examples, random_dim])
    generated_images = generator.predict(noise)
    generated_images = generated_images.reshape(examples, 28, 28)
    plt.figure(figsize=figsize)

    for i in range(generated_images.shape[0]):
        plt.subplot(dim[0], dim[1], i+1)
        plt.imshow(generated_images[i], interpolation='nearest', cmap='gray_r')
        plt.axis('off')
    plt.tight_layout()
    plt.savefig('gan_generated_image_epoch_%d.png' % epoch)

In [ ]:
def train(epochs=1,batch_size=128):      #model training process
    x_train, y_train, x_test, y_test = load_dataset() #get the data
    batchCount = x_train.shape[0]//128 #split the training data into batches
    #built GAN network
    generator=Generator(optimizer)
    discriminator=Discriminator(optimizer)
    GAN=model(generator, discriminator, optimizer, random_dim)
    
    for e in range(1, epochs+1):
        print ('-'*15, 'Epoch %d' % e, '-'*15)
        for _ in tqdm(range(batchCount)):
            # Get a random set of input noise and images
            noise = np.random.normal(0, 1, size=[128, random_dim])
            image_batch = x_train[np.random.randint(0, x_train.shape[0], size=128)]

            # Generate fake MNIST images
            generated_images = generator.predict(noise)
            X = np.concatenate([image_batch, generated_images])

            # Labels for generated and real data
            y_dis = np.zeros(2*128)
            # One-sided label smoothing
            y_dis[:128] = 0.9

            # Train discriminator
            discriminator.trainable = True
            discriminator.train_on_batch(X, y_dis)

            # Train generator
            noise = np.random.normal(0, 1, size=[128, random_dim])
            y_gen = np.ones(128)
            discriminator.trainable = False
            GAN.train_on_batch(noise, y_gen)
            
    if e==1 or e%20 == 0:
       plot_generated_images(e, generator)

if __name__ == '__main__':
    train(50, 128)  

--------------- Epoch 1 ---------------


  0%|          | 0/468 [00:00<?, ?it/s]

4/4 [==============================] - 0s 15ms/step


  0%|          | 1/468 [00:02<16:55,  2.18s/it]

4/4 [==============================] - 0s 7ms/step


...

4/4 [==============================] - 0s 13ms/step


 13%|█▎        | 60/468 [00:18<02:01,  3.36it/s]

4/4 [==============================] - 0s 10ms/step


 13%|█▎        | 61/468 [00:19<02:06,  3.22it/s]

4/4 [==============================] - 0s 10ms/step
